# Experimental on Sound Event Classification.

We were training a deep CNN with only 2k data. The CNN tries to classificate this data into 5 different classes and it has a poor result. It would be more efficient to classify the sounds randomly.

This problem combined with the huge cpu-consuming of train a CNN moves us into a cul-de-sac. 

But there's a glimmer of hope. And that's what we are going to try here.

Our main objective here is the following: To reduce the complexity of the model by creating an image based on the mean of intensity in each frequence.

In [1]:
import librosa
import librosa.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re

A bit of what we are going to try.

In [2]:
dat, sr = librosa.load('../Datasets/s_a_d__datasets/audio_esc50/1-100210-B-36.wav')

In [3]:
arr = librosa.feature.melspectrogram(y=dat, sr=sr)

In [4]:
arr.shape

(128, 216)

In [5]:
arr

array([[8.5993549e+02, 4.5912991e+02, 2.6354221e+02, ..., 2.5969156e+01,
        1.8569384e+01, 8.6176844e+00],
       [5.7142612e+01, 2.9574568e+01, 2.2764557e+01, ..., 3.1396091e+00,
        3.3211610e+00, 8.9324789e+00],
       [1.5904705e+01, 4.5833412e+01, 4.8435863e+01, ..., 5.5756366e-01,
        6.3307798e-01, 7.3707592e-01],
       ...,
       [1.1063246e+00, 1.2394960e+00, 1.4005175e+00, ..., 1.1156948e-02,
        7.8027854e-03, 4.8842449e-03],
       [1.9012589e-02, 1.2849880e-02, 9.9226097e-03, ..., 2.2470423e-03,
        1.4463906e-03, 1.5137937e-03],
       [1.7806103e-02, 4.8770974e-03, 6.3218613e-04, ..., 1.5918356e-04,
        1.8024274e-04, 3.4940013e-04]], dtype=float32)

In [6]:
algo = np.mean(arr.T, axis=0)

In [7]:
algo.shape

(128,)

In [8]:
algo = np.reshape(algo, (16, 8))

In [9]:
algo

array([[1.8423973e+02, 9.5221672e+01, 4.1524628e+01, 3.1253235e+01,
        1.1174012e+01, 4.7411294e+00, 2.9177074e+00, 3.7878728e+00],
       [4.1952629e+00, 4.0654149e+00, 5.6711202e+00, 7.9348154e+00,
        8.6790867e+00, 5.7510371e+00, 7.8876605e+00, 7.0596685e+00],
       [8.1930952e+00, 6.9106083e+00, 6.9541159e+00, 5.4830785e+00,
        1.8071291e+01, 3.2125507e+01, 8.2721148e+00, 5.9156137e+00],
       [4.7791600e+00, 4.8593578e+00, 4.6287632e+00, 3.1341968e+00,
        2.0397530e+00, 2.2277358e+00, 4.3026896e+00, 4.6487260e+00],
       [4.0277691e+00, 3.7509263e+00, 4.3718848e+00, 2.9770634e+00,
        3.8824282e+00, 5.6923251e+00, 5.4853811e+00, 5.4030142e+00],
       [6.5696130e+00, 1.0472785e+01, 4.7467525e+01, 1.0835682e+01,
        6.4205809e+00, 6.2058992e+00, 5.6079807e+00, 4.0754209e+00],
       [2.5316885e+00, 1.8539383e+00, 1.6581200e+00, 2.1909628e+00,
        2.3071566e+00, 1.6815077e+00, 1.4791418e+00, 1.2806550e+00],
       [1.3038816e+00, 1.5726756e+00, 1.4

### Scaling of the idea.

In [10]:
def create_melspec(params, audio_data, sampling_rate):
    S = librosa.feature.melspectrogram(audio_data, sr=sampling_rate, n_mels=params['n_mels'],
                                      hop_length=params['hop_length'], n_fft=params['n_fft'],
                                      fmin=params['fmin'], fmax=(sampling_rate//2))
    S_dB = librosa.power_to_db(S, ref=np.max)
    S_dB = S_dB.astype(np.float32)

    return S

In [11]:
def load_audio(single_file):
    # Take in count that our dataset files lasts everyone of them the same time. If not you should do something.
    audio, sr = librosa.load(single_file, duration=5.0)
    return audio, sr

In [12]:
    melspec_params = {
        'n_mels': 128, # The entire frequency spectrum divided by a concrete number.
        'duration': 5*22050, # Number of seconds * sample rate
        'hop_length': 512, # It has something to do with the duration. I think it fills the space with repetitions
        'n_fft': 2048, # Length of the Fast Fourier Transformation
        'fmin': 20
}

In [13]:
files_list = os.listdir('../Datasets/s_a_d__datasets/audio_esc50')
paths_list = []
for file in files_list:
    paths_list.append('../Datasets/s_a_d__datasets/audio_esc50/'+file)

In [14]:
for file, path in zip(files_list, paths_list):
    audio_file, sr = load_audio(path)
    melspec = create_melspec(melspec_params, audio_file, sr)
    df = pd.DataFrame(melspec)
    df.to_csv('../Datasets/s_a_d__datasets/experimental/' + file[:-4]+'.csv')

## The brand new Neural Network.

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from keras.callbacks import EarlyStopping

In [15]:
files_list_csv = os.listdir('../Datasets/s_a_d__datasets/experimental')

In [16]:
Images = []
Labels = []

for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/'+file)
    Images.append(np.mean(image_to_be.to_numpy().T, axis=0))
    Labels.append(re.split(r'-|\.', file)[3])    

In [17]:
Images = np.asarray(Images)
Labels = np.asarray(Labels)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Parentesis. Try a LogisticRegression.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

In [22]:
logisticreg = LogisticRegression()

In [23]:
logisticreg.fit(X_train, y_train)

/home/toni_domenech/Python/envs/s_a_d/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
predictions = logisticreg.predict(X_test)

In [27]:
score = logisticreg.score(X_test, y_test)
print(score)

0.16666666666666666


In [80]:
X_train.shape

(1400, 128)

In [81]:
X_train = X_train.reshape(1400, 16, 8, 1)
X_test = X_test.reshape(600, 16, 8, 1)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

input_dim = (16,8,1)

In [82]:
model = Sequential()

In [83]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [84]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=8, verbose=1, mode='auto')]

In [85]:
model.fit(X_train, y_train, epochs=50, batch_size=50)

Epoch 1/50
28/28 [==============================] - 0s 15ms/step - loss: 3.4732 - accuracy: 0.1036
Epoch 2/50
28/28 [==============================] - 0s 15ms/step - loss: 2.7136 - accuracy: 0.2464
Epoch 3/50
28/28 [==============================] - 0s 15ms/step - loss: 2.4091 - accuracy: 0.3107
Epoch 4/50
28/28 [==============================] - 0s 15ms/step - loss: 2.1096 - accuracy: 0.3857
Epoch 5/50
28/28 [==============================] - 0s 15ms/step - loss: 1.8854 - accuracy: 0.4514
Epoch 6/50
28/28 [==============================] - 0s 16ms/step - loss: 1.6937 - accuracy: 0.5036
Epoch 7/50
28/28 [==============================] - 0s 16ms/step - loss: 1.5026 - accuracy: 0.5629
Epoch 8/50
28/28 [==============================] - 0s 16ms/step - loss: 1.3507 - accuracy: 0.6121
Epoch 9/50
28/28 [==============================] - 0s 15ms/step - loss: 1.2433 - accuracy: 0.6436
Epoch 10/50
28/28 [==============================] - 0s 16ms/step - loss: 1.1303 - accuracy: 0.6829
Epoch 11/

In [86]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 0s 3ms/step - loss: 4.0021 - accuracy: 0.4283
[4.002133846282959, 0.4283333420753479]


**We have clearly overfitted our data. We can see it in the difference between the accuracy on the train set and the one in the test set.**

In order to solve this we can do two different things. Simplify the model or use more data. We are going to multiply the size of our data.

In our first simplification of the problem we did the mean on the intensity axis only. So we had eliminated the time one. It's ok if we only see the intensity but it's worth nothing that the sound have an order in space. So if we do little frames of time we could do a growth in data.

### The idea:

In [87]:
dat, sr = librosa.load('../Datasets/s_a_d__datasets/audio_esc50/1-100210-B-36.wav')

In [88]:
arr = librosa.feature.melspectrogram(y=dat, sr=sr)

In [89]:
algo_1 = np.mean(arr[:,:128].T, axis=0)

In [90]:
algo_1.shape

(128,)

In [91]:
algo_2 = np.mean(arr[:,128:].T, axis=0)

In [92]:
def_algo = []
for i in range (len(algo_1)):
    def_algo.append(algo_1[i])
    def_algo.append(algo_2[i])

In [120]:
def_algo = np.asarray(def_algo)

### Scaling the idea:

In [16]:
Images = []
Labels = []

for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/' + file)
    image_to_be = np.asarray(image_to_be)
    first_array = np.mean(image_to_be[:,:128].T, axis=0)
    second_array = np.mean(image_to_be[:,128:].T, axis=0)
    def_array = []
    for i in range(len(first_array)):
        def_array.append(first_array[i])
        def_array.append(second_array[i])
    Images.append(def_array)
    Labels.append(re.split(r'-|\.', file)[3])    

In [20]:
Images = np.asarray(Images)
Labels = np.asarray(Labels)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

In [22]:
X_train = X_train.reshape(1400, 16, 16, 1)
X_test = X_test.reshape(600, 16, 16, 1)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

input_dim = (16,16,1)

In [25]:
model = Sequential()

In [26]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=8, verbose=1, mode='auto')]

In [28]:
model.fit(X_train, y_train, epochs=50, batch_size=50)

Epoch 1/50
28/28 [==============================] - 1s 28ms/step - loss: 3.3552 - accuracy: 0.1371
Epoch 2/50
28/28 [==============================] - 1s 29ms/step - loss: 2.5247 - accuracy: 0.3007
Epoch 3/50
28/28 [==============================] - 1s 29ms/step - loss: 2.1177 - accuracy: 0.4000
Epoch 4/50
28/28 [==============================] - 1s 29ms/step - loss: 1.7681 - accuracy: 0.4921
Epoch 5/50
28/28 [==============================] - 1s 29ms/step - loss: 1.5601 - accuracy: 0.5379
Epoch 6/50
28/28 [==============================] - 1s 29ms/step - loss: 1.3676 - accuracy: 0.6086
Epoch 7/50
28/28 [==============================] - 1s 29ms/step - loss: 1.2004 - accuracy: 0.6479
Epoch 8/50
28/28 [==============================] - 1s 29ms/step - loss: 1.0820 - accuracy: 0.6886
Epoch 9/50
28/28 [==============================] - 1s 29ms/step - loss: 0.9445 - accuracy: 0.7307
Epoch 10/50
28/28 [==============================] - 1s 29ms/step - loss: 0.8478 - accuracy: 0.7650
Epoch 11/

In [29]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 0s 5ms/step - loss: 4.8380 - accuracy: 0.4083
[4.83804988861084, 0.40833333134651184]


We can see that we haven't imporved too much (or nothing) our previous result. But that's because there aren't many sounds lasting 2.5 seconds. Maybe if we can zoom in the sound (using 4, 8 or 16 cuts instead of 1) the result will improve.

In [17]:
def audio_cuts(number_of_cuts, list_of_files):
    cut_size = 216//number_of_cuts
    Images = []
    Labels = []
    for file in list_of_files:
        image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/' + file)
        image_to_be = np.asarray(image_to_be)
    
        cuts = [[] for i in range(number_of_cuts)]
        
        for  i in range(number_of_cuts):
            cuts[i] = np.mean(image_to_be[:,i*cut_size:(i+1)*cut_size].T, axis=0)
    
        def_array = []
        for i in range(len(cuts[0])):
            for j in range(len(cuts)):
                def_array.append(cuts[j][i])
        Labels.append(re.split(r'-|\.', file)[3])
        Images.append(def_array)
    Images = np.asarray(Images)
    Labels = np.asarray(Labels)
    Labels = pd.get_dummies(Labels)
    return Images, Labels

In [18]:
Images, Labels = audio_cuts(8, files_list_csv)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

In [22]:
X_train = X_train.reshape(1400, 32, 32, 1)
X_test = X_test.reshape(600, 32, 32, 1)


input_dim = (32,32,1)

In [41]:
model = Sequential()

In [42]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh"))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [43]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])


In [44]:
model.fit(X_train, y_train, epochs=50, batch_size=50, validation_split=0.3)

Epoch 1/8
20/20 [==============================] - 5s 229ms/step - loss: 3.6070 - accuracy: 0.0899 - val_loss: 3.0140 - val_accuracy: 0.1924
Epoch 2/8
20/20 [==============================] - 4s 223ms/step - loss: 2.4983 - accuracy: 0.2993 - val_loss: 2.4658 - val_accuracy: 0.3088
Epoch 3/8
20/20 [==============================] - 5s 251ms/step - loss: 1.8483 - accuracy: 0.4719 - val_loss: 2.5308 - val_accuracy: 0.3088
Epoch 4/8
20/20 [==============================] - 5s 255ms/step - loss: 1.3889 - accuracy: 0.5812 - val_loss: 2.4498 - val_accuracy: 0.3824
Epoch 5/8
20/20 [==============================] - 5s 253ms/step - loss: 1.0697 - accuracy: 0.6691 - val_loss: 2.5093 - val_accuracy: 0.3658
Epoch 6/8
20/20 [==============================] - 5s 259ms/step - loss: 0.8767 - accuracy: 0.7508 - val_loss: 2.4830 - val_accuracy: 0.3753
Epoch 7/8
20/20 [==============================] - 5s 252ms/step - loss: 0.7532 - accuracy: 0.7732 - val_loss: 2.4807 - val_accuracy: 0.3919
Epoch 8/8
20/

In [45]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 1s 35ms/step - loss: 3.0352 - accuracy: 0.3467
[3.0351672172546387, 0.3466666638851166]


### Passing the images complete to a simpler architecture.

In [16]:
Images = []
Labels = []

for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/'+file)
    Images.append(image_to_be.to_numpy())
    Labels.append(re.split(r'-|\.', file)[3])

In [17]:
Images = np.asarray(Images)
Labels = np.asarray(Labels)

In [18]:
Labels = pd.get_dummies(Labels)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

NameError: name 'train_test_split' is not defined

In [ ]:
X_train = X_train.reshape(1400, 128, 217, 1)
X_test = X_test.reshape(600, 128, 217, 1)


input_dim = (128,217,1)

In [ ]:
input_dim = ([128, 217, 3])

In [ ]:
from keras.layers import GlobalAveragePooling2D

In [50]:
model = Sequential()

In [51]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = (128, 217, 1)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=8, verbose=1, mode='auto')]

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 128, 217, 64)      640       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 217, 64)      36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 108, 64)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 108, 128)      73856     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 64, 108, 128)      147584    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 54, 128)       0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 128)              

In [54]:
model.fit(X_train, y_train, epochs=50, batch_size=36, callbacks=callbacks)

Epoch 1/50
39/39 [==============================] - 159s 4s/step - loss: 3.7389 - accuracy: 0.0343
Epoch 2/50
39/39 [==============================] - 171s 4s/step - loss: 3.4418 - accuracy: 0.0586
Epoch 3/50
39/39 [==============================] - 201s 5s/step - loss: 3.2350 - accuracy: 0.1064
Epoch 4/50
39/39 [==============================] - 185s 5s/step - loss: 2.9914 - accuracy: 0.1371
Epoch 5/50
39/39 [==============================] - 190s 5s/step - loss: 2.7733 - accuracy: 0.1886
Epoch 6/50
39/39 [==============================] - 188s 5s/step - loss: 2.6648 - accuracy: 0.2057
Epoch 7/50
39/39 [==============================] - 202s 5s/step - loss: 2.5613 - accuracy: 0.2207
Epoch 8/50
39/39 [==============================] - 208s 5s/step - loss: 2.4907 - accuracy: 0.2407
Epoch 9/50
39/39 [==============================] - 179s 5s/step - loss: 2.4679 - accuracy: 0.2486
Epoch 10/50
39/39 [==============================] - 179s 5s/step - loss: 2.4013 - accuracy: 0.2700
Epoch 11/

In [55]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 19s 982ms/step - loss: 1.8190 - accuracy: 0.5000
[1.8190422058105469, 0.5]


In [61]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [76]:
y_pred = np.argmax(predictions, axis=1)

confusion_matrix(y_test ,y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [30000, 600]

In [65]:
y_pred = np.argmax(predictions, axis=1)

In [66]:
y_pred

array([ 9, 24, 21, 37,  9, 47,  8, 13,  9, 32, 14, 43,  1, 24, 46,  3, 40,
       17, 30, 43, 20, 44, 24, 14, 31,  1, 24, 47,  1, 25, 26, 37, 20, 21,
       37, 24,  0,  2, 28, 34, 14, 34, 44,  3, 44, 29, 18, 40,  5,  3, 20,
       36, 12, 46, 18, 24, 17,  6, 10, 37, 30, 37,  8, 44, 25,  1, 15,  1,
       30, 46, 48, 35, 43, 33, 31, 39,  2, 46, 37, 14, 26, 36, 22, 37,  6,
       12, 43, 11, 25, 26,  9,  1, 11,  4, 17, 14,  1, 20, 30, 11, 10, 37,
       14, 11, 10, 39,  1, 39, 39, 40, 30, 47, 12, 29, 14, 33, 19, 23, 46,
       13, 22, 47, 15, 25,  2, 44, 30,  5,  1, 43, 37, 24,  6, 18, 28, 39,
        4, 31, 36, 47,  7,  9, 39, 24, 37, 32, 37, 26, 25, 24, 20,  6, 19,
        2, 36, 43, 24, 46, 47,  9, 17, 32, 26, 33, 43,  6,  9, 37,  7, 29,
       21, 10, 15,  4, 34,  2, 39,  1, 25, 40, 32, 32, 37, 18, 14, 37, 21,
       17, 47,  9, 16, 21, 10, 39, 46, 44, 11, 34, 25, 18,  4, 17, 19, 25,
       47, 24, 21, 29, 34,  1,  6, 10, 21, 30, 24,  1, 22, 13, 43, 18,  1,
       17, 21,  1, 18, 20

In [75]:
y_test = y_test.stack().reset_index().drop(0,1)['level_1'].to_numpy()

### Now it's time to paramter tunning.

First of all we are going to try it with different activation funcitons.

In [24]:
model = Sequential()

In [25]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [27]:
model.fit(X_train, y_train, epochs=50, batch_size=36, callbacks=callbacks)

Epoch 1/50
39/39 [==============================] - 146s 4s/step - loss: 3.8624 - accuracy: 0.0286
Epoch 2/50
39/39 [==============================] - 146s 4s/step - loss: 3.7397 - accuracy: 0.0550
Epoch 3/50
39/39 [==============================] - 188s 5s/step - loss: 3.6024 - accuracy: 0.0736
Epoch 4/50
39/39 [==============================] - 191s 5s/step - loss: 3.4642 - accuracy: 0.0950
Epoch 5/50
39/39 [==============================] - 199s 5s/step - loss: 3.2238 - accuracy: 0.1164
Epoch 6/50
39/39 [==============================] - 181s 5s/step - loss: 3.0859 - accuracy: 0.1443
Epoch 7/50
39/39 [==============================] - 184s 5s/step - loss: 3.0260 - accuracy: 0.1607
Epoch 8/50
39/39 [==============================] - 181s 5s/step - loss: 2.8837 - accuracy: 0.1993
Epoch 9/50
39/39 [==============================] - 209s 5s/step - loss: 2.7711 - accuracy: 0.1814
Epoch 10/50
39/39 [==============================] - 354s 9s/step - loss: 2.6929 - accuracy: 0.2350
Epoch 11/

In [28]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 17s 904ms/step - loss: 2.6138 - accuracy: 0.5550
[2.613813877105713, 0.5550000071525574]
